# Lesson 1: Evaluate and plot surrogates

In [ ]:
### setup system path ###
# NOTE: This step is NOT needed if gwsurrogate was installed from pip, with "python setup.py install" 
# or if PYTHONPATH has been set (see README)
import sys, os
path_to_gws = '/home/scott/Repos/GitRepos/Codes/gwsurrogate/'
sys.path.append(path_to_gws)

In [ ]:
### import the gwsurrogate and gwtools modules and load an existing surrogate ###
import gwsurrogate as gws
import gwsurrogate.gwtools as gwtools

# assumes notebook launched from "notebooks" directory
path_to_surrogate = path_to_gws+'tutorial/TutorialSurrogate/EOB_q1_2_NoSpin_Mode22/l2_m2_len12239M_SurID19poly/'
EOBNRv2_sur       = gws.EvaluateSingleModeSurrogate(path_to_surrogate) # multi-mode evaluation covered in other notebooks

### import other useful packages ###
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
### evaluate and plot an EOB surrogate waveform (physical) ###
t, hp, hc  = EOBNRv2_sur(q=1.7, M=80.0, dist=1.0, phi_ref = 0.0, f_low = 10.0)
amp, phase = EOBNRv2_sur.amp_phase(hp + 1j*hc)

gwtools.plot_pretty(t, [hp, hc])
plt.plot(t,amp,'r')
plt.xlabel('t (seconds)')
plt.show()

In [ ]:
### Variety of helper functions, for example, compute the mode's phase at merger...
phi_m = EOBNRv2_sur.phi_merger(hp + 1j*hc)
print 'current phase is '+str( np.mod(EOBNRv2_sur.phi_merger(hp + 1j*hc),2.0*np.pi) )

### ...modify h, without generating a new waveform, such that the phase is now 2.0
h_adj = EOBNRv2_sur.adjust_merger_phase(hp + 1j*hc,2.0)
print 'new phase is '+str( np.mod(EOBNRv2_sur.phi_merger(h_adj),2.0 * np.pi) )

In [ ]:
### direct plotting is also supported (dimensionless) ###
EOBNRv2_sur.plot_sur(1.7);

In [ ]:
### if you try to evaluate the surrogate outside of its training range, an error is thrown ###
t1, hp1, hc1 = EOBNRv2_sur(q=2.7, M=80.0, dist=1.0)

# Lesson 2: Sampling the surrogate at different times

In [ ]:
# Surrogate at q=1.2 with original time sampling (i.e. the one for which it was built)
t, hp, hc = EOBNRv2_sur(q=1.2)

In [ ]:
# Surrogate at q=1.2 with 3000 samples in an enlarged interval of time
# Sampling outside of the training time interval returns zeros
t_resamp, hp_resamp, hc_resamp = EOBNRv2_sur(1.2,samples=np.linspace(EOBNRv2_sur.tmin-1000,EOBNRv2_sur.tmax+1000,num=3000))
EOBNRv2_sur.plot_pretty(t, [hp, hc],1)
EOBNRv2_sur.plot_pretty(t_resamp, [hp_resamp, hc_resamp],2)
EOBNRv2_sur.plt.show()

# Lesson 3: Surrogate properties

In [ ]:
### Greedy points and eim indicies ###
t, hp, hc = EOBNRv2_sur(1.7,80.0,1.0)

eim_pts    = EOBNRv2_sur.eim_indices
T_eim      = t[eim_pts]
greedy_pts = EOBNRv2_sur.greedy_points

EOBNRv2_sur.plot_pretty(t, [hp, hc])
plt.plot(T_eim,np.zeros(eim_pts.shape),'r*',markersize=15)
plt.show()

print 'Mass ratios defining the basis: ', greedy_pts

In [ ]:
### Temporal information is stored with dimensionless 't/M' units ###
print 'temporal units is '+EOBNRv2_sur.t_units
print 'temporal sampling is '+str(EOBNRv2_sur.dt)
print 'sampling rate is '+str(1.0/EOBNRv2_sur.dt)
print 'initial time is '+str(EOBNRv2_sur.tmin)
print 'final time is '+str(EOBNRv2_sur.tmax)

# Lesson 4: Surrogate basis functions

We now show two ways of recovering the 5th physical RB waveform. The first way evalutes the surrogate at the 5th greedy point q_5. The second way uses the Vandermonde matrix V and the GS matrix R (defined as A = QR) to exactly recover the 5th physical RB waveform. Having two (kinda) independent ways to compute h_5 provides a check of R,V, and the surrogate model error at this value of q.

In [ ]:
### plot the 5th cardinal (B) basis which appears as h_surrogate = Sum_i B_i(t) h(T_i)  ###
times = EOBNRv2_sur.time()
b_5   = EOBNRv2_sur.basis(4,'cardinal')

EOBNRv2_sur.plot_pretty(times,[b_5.real,b_5.imag],1)
plt.title('Cardinal basis')

### use the matrix V, as E = B V, to exactly recover the 5th orthonormal basis E ###
e_5 = EOBNRv2_sur.basis(4,'orthogonal')

EOBNRv2_sur.plot_pretty(times,[e_5.real,e_5.imag],2)
plt.title('Orthogonal basis')

### use the matrix R, as H = E R, to recover the 5th physical basis H ###
h_5 = EOBNRv2_sur.basis(4,'waveform')

EOBNRv2_sur.plot_pretty(times,[h_5.real,h_5.imag],3)
plt.title('Waveform basis')

### use the surrogate model to evaluate for the 5th physical basis ###
junk, hp_5_surr, hc_5_surr = EOBNRv2_sur(EOBNRv2_sur.greedy_points[4])
nrm_5 = EOBNRv2_sur.norm_eval(EOBNRv2_sur.greedy_points[4]) # normalization constant for comparison with normalized basis
hp_5_surr = hp_5_surr / nrm_5 
hc_5_surr = hc_5_surr / nrm_5
h_5_surr = hp_5_surr + 1j*hc_5_surr

EOBNRv2_sur.plot_pretty(times,[hp_5_surr,hc_5_surr],4)
plt.title('Waveform basis via surrogate model')

### plot the difference between the 5th physical basis and its surrogate -- consistency and error check ###
EOBNRv2_sur.plot_pretty(times,[h_5.real - hp_5_surr,h_5.imag - hc_5_surr],5,'semilogy')
plt.title('Difference between physical waveform basis and its surrogate prediction')

plt.show()

In [ ]:
### check basis orthonormality (using Euclidean inner product) ###
### NOTE: Basis orthogonal on physical grid (in seconds). See surrogate's info.data file
dt  = 1.0/2048.0
e_6 = EOBNRv2_sur.basis(5,'orthogonal')
print '<e_5,e_6> = ', np.sum(e_5*np.conj(e_6)) * dt
print '<e_5,e_5> = ', np.sum(e_5*np.conj(e_5)) * dt

In [ ]:
### plotting the waveform basis using the surrogate is also a built-in function ###
EOBNRv2_sur.plot_rb(4)

# Lesson 5: Download surrogates from a server

Surrogate may be located on a server. gwsurrogate provides a list of the locations as well as download tools

In [ ]:
### view all available surrogates ###
gws.surrogate_repo.list()

In [ ]:
### display the default location for all surrogate downloads ###
gws.surrogate_repo.download_path()

In [ ]:
### download a surrogate (this will take a moment, check the terminal for progress) ##
surr_path = gws.surrogate_repo.get('SpEC_q1_10_NoSpin')

# ...or specify a custom download location as 
#surr_path = gws.surrogate_repo.get('SpEC_q1_10_NoSpin','MY_PATH')

In [ ]:
# load the multimode numerical relativity surrogate you've just downloaded #
spec = gws.EvaluateSurrogate(surr_path)

In [ ]:
# evaluate and plot a surrogate #
t, hp, hc = spec(q=3.14,ell=[2],m=[2],fake_neg_modes=False)

spec.plot_pretty(t,[hp,hc],showQ=False)
plt.plot(t,np.abs(hp+1.0j*hc),'blue')
plt.xlabel('$t/M$')
plt.title('(2,2) mode for q = 3.14')
plt.show()